# Annexes 

## Table des matières

* [Codes supplémentaires](#section1)
    * [Multi-Threading](#section11)
    * [Travail sur le poids des fichiers](#section12)
        * [Zip](#section121)
        * [Partition des fichiers](#section122)
* [Nettoyage des données de Légifrance](#section2)
    * [xxx](#section21)
* [Sauvegarde des tableaux de données finalisées](#section3)


## Codes supplémentaires<a class="anchor" id="section1"></a>

Dans cette section seront regroupés tous les codes qui ont été faits, en partie ou totalement, mais qui ne sont finalement pas intégrés au projet final. Nous les avons mis car ils sont potentiellement intéressants, car explorant d'autre méthodes par exemple, et sont en lien avec le projet, donc pourraient servir à l'avenir dans des projets similaires. 

### Multi-Threading <a class="anchor" id="section11"></a>

Lors de nos tentatives de récupération des données via l'API Piste, une méthode envisagée fût de récupérer les fichiers sur des pages de de taille 1, car cela semblait fonctionner au delà du 10001e éléments. Ainsi pour pallier les contraintes de temps du projet nous avons codé de quoi faire simultanément les récupération de données en multi-threading. 

L'idée est donc de calculer le nombre de pages restantes pour la requête souhaitée, puis en donnant le nombre de threads souhaités produire le même nombre de fonctions. Ces fonctions récupéreront les données sur des plages de pages de tailles similaires, les plages correspondent environ au ratio de pages restantes par rapport aux nombres de fonctions. 

In [ ]:
def remaining_page_number(file_name):
    '''
    Calcule le nombre de pages restantes à récupérer en comparant le nombre total de résultats et la progression sauvegardée dans un fichier spécifique.

    :param file_name: Nom du fichier JSON utilisé pour sauvegarder la progression (sans l'extension).
    :return: Le nombre de pages restantes à récupérer à partir de la progression sauvegardée.
    '''
    api_host = API_HOST+"/search"
    client = get_client()
    response = client.post(api_host, json=code_api).json()
    total_results = response.get("totalResultNumber", 0)
    file_name = str(file_name)+".json"
    try:
        # Charger les données existantes si le fichier existe
        with open(file_name, "r", encoding="utf-8") as file:
            existing_data = json.load(file)
            if not isinstance(existing_data, dict):
                raise ValueError("Le fichier de sauvegarde n'est pas correctement structuré.")
            start_page = existing_data.get("current_page", 1)
    except (FileNotFoundError, ValueError):
        start_page = 0
    remaining_page = total_results-start_page
    return remaining_page    

In [ ]:
def collect_all_results_between(api_host, code, page_to_start, page_to_end, thread_number):
    '''
    Récupère les résultats d'une requête API sur une plage spécifique de pages.

    :param api_host: Adresse du serveur où envoyer la requête avec le endpoint correspondant.
    :param code: Code de la requête en json
    :param page_to_start: Numéro de la première page à récupérer dans la plage.
    :param page_to_end: Numéro de la dernière page à récupérer dans la plage.
    :param thread_number: Numéro du thread utilisé pour différencier les fichiers de sauvegarde.
    :return: Aucun retour direct, les résultats sont sauvegardés dans un fichier JSON.
    '''

    client = get_client()
    expires_in = 55*60
    token_expiry = datetime.now() + timedelta(seconds=expires_in)

    file_name = str(thread_number)+"results.json"


    all_results = []

    for page_number in range(page_to_start, page_to_end + 1):
        # Vérifier si le token doit être renouvelé
        if datetime.now() >= token_expiry:
            print("Renouvellement du client OAuth...")
            client = get_client()
            token_expiry = datetime.now() + timedelta(seconds=expires_in)

        print(f"Récupération de la page {page_number}/{page_to_end - page_to_start +1}...")
        code["recherche"]["pageNumber"] = page_number
        response = client.post(api_host, json=code).json()
        page_results = response.get("results", [])

        if response.get("error") == 503:
            print(response)
            break

        if page_number % 10 == 0: 
            print(response)

        # Ajouter les résultats de la page courante
        all_results.extend(page_results)

        # Sauvegarder les résultats toutes les 20 pages ou à la dernière page
        if page_number % 20 == 0 or page_number == page_to_end:
            print(f"Ajout des pages jusqu'à la page {page_number} dans {file_name}...")
            save_results_to_file(all_results, file_name, page_number)

            # Réinitialiser la liste des résultats sauvegardés
            all_results = []


In [ ]:
def generate_functions(n, file_name):
        '''
        Génère un ensemble de fonctions pour récupérer des pages de résultats en parallèle.

        :param n: Nombre de threads ou de fonctions à générer.
        :param file_name: Nom du fichier JSON utilisé pour sauvegarder la progression (sans l'extension).
        :return: Un dictionnaire contenant les fonctions générées, prêtes à être exécutées pour traiter une plage de pages.
        '''

        api_host = API_HOST+"/search"
        client = get_client()
        remaining_page = remaining_page_number(file_name)
        functions = {}
        file_name = str(file_name)+".json"
        response = client.post(API_HOST+"/search", json=code_api).json()
        total_results = response.get("totalResultNumber", 0)
        try:
            with open(file_name, "r", encoding="utf-8") as file:
                existing_data = json.load(file)
                if not isinstance(existing_data, dict):
                    raise ValueError("Le fichier de sauvegarde n'est pas correctement structuré.")
                start_page = existing_data.get("current_page", 1)
        except (FileNotFoundError, ValueError):
            start_page = 1

        page_state = [start_page, start_page]

        for i in range(1, n + 1):
            if i != n : 
                    page_to_end = [int(np.floor(remaining_page*i/n)+ page_state[1])]
            else : 
                    page_to_end = [total_results]

            thread_number = i

            def func_template(idx=i, start=page_state[0], end=page_to_end[0], thread_nbr= thread_number):
                collect_all_results_between(API_HOST+"/search", code_api, start, end, thread_nbr)

            page_state[0]= page_to_end[0] + 1
                
            functions[f"f_{i}"] = func_template
        return functions

In [ ]:
def functions_to_thread(n, file_name):
    '''
    Prépare une liste de fonctions générées avec leurs arguments pour une exécution dans des threads.

    :param n: Nombre de threads ou de fonctions à générer.
    :param file_name: Nom du fichier JSON utilisé pour sauvegarder la progression (sans l'extension).
    :return: Une liste de tuples contenant les fonctions générées, leurs arguments (liste vide), et leurs mots-clés (dictionnaire vide).
    '''

    generated_functions = generate_functions(n, file_name)
    functions = [(generated_functions[f"f_{i}"], [], {}) for i in range(1, n+1) ]
    return functions

In [ ]:
def run_in_threads(functions):
    '''
    Exécute un ensemble de fonctions dans des threads distincts.

    :param functions: Liste de tuples contenant les fonctions à exécuter, leurs arguments (sous forme de liste), et leurs mots-clés (sous forme de dictionnaire).
    :return: Aucun retour direct. Les threads sont lancés et exécutés, puis attendus jusqu'à leur complétion.
    '''

    threads = []

    for func, args, kwargs in functions:
        thread = Thread(target=func, args=args, kwargs=kwargs)
        threads.append(thread)
        thread.start()

    # Attendre la fin de tous les threads
    for thread in threads:
        thread.join()

### Travail sur le poids des fichiers <a class="anchor" id="section12"></a>

Lors de la récolte des données de Légifrance nous avons fait face à des fichiers très lourds, parfois proche du giga, dès lors deux choses ont été entreprises plus ou moins partiellement. 

#### Zip <a class="anchor" id="section121"></a>

La première méthode et la plus classique consistait en une compression des fichiers au format Zip, ce qui se révéla très peu efficace car les fichiers json sont déjà bien compressés par essence. 

In [ ]:
def zipper_fichier(fichier, zip_nom):
    """
    Crée un fichier ZIP contenant le fichier spécifié et supprime le fichier d'origine

    :param fichier: Chemin du fichier à zipper.
    :param zip_nom: Nom du fichier ZIP de sortie.
    """
    with zipfile.ZipFile(zip_nom, 'w') as zipf:
        zipf.write(fichier, arcname=fichier.split('/')[-1]) 
        os.remove(fichier)

In [ ]:
def extraire_json_du_zip(fichier_zip, fichier_sortie):
    """
    Extrait un fichier JSON contenu dans une archive ZIP et le sauvegarde.

    :param fichier_zip: Chemin de l'archive ZIP contenant le fichier JSON.
    :param fichier_sortie: Chemin du fichier JSON extrait.
    """
    with zipfile.ZipFile(fichier_zip, 'r') as zipf:
        json_fichier = zipf.namelist()[0]  
        with zipf.open(json_fichier) as file:
            data = json.load(file)
        with open(fichier_sortie, 'w', encoding='utf-8') as json_file:
            json.dump(data, json_file, indent=4)

##### Partition des fichiers <a class="anchor" id="section122"></a>

La seconde méthode est restée au stade de brouillon mais consistait en une partition des fichier json afin de pouvoir les envoyer segmentés sur github avant de les reconstruire au moment de l'exécution des fonctions.

In [ ]:
def segmenter_json_par_parties(fichier_json, dossier_sortie, nombre_parties):
    """
    Segmente un fichier JSON en un nombre spécifique de parties.

    :param fichier_json: Chemin du fichier JSON à segmenter.
    :param dossier_sortie: Dossier où les segments seront sauvegardés.
    :param nombre_parties: Nombre de parties dans lesquelles le fichier sera segmenté.
    """
    # Lire le contenu du fichier JSON
    with open(fichier_json, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Calculer la taille approximative de chaque partie
    taille_segment = math.ceil(len(data) / nombre_parties)

    # Créer le dossier de sortie s'il n'existe pas
    os.makedirs(dossier_sortie, exist_ok=True)

    # Segmenter les données
    for i in range(0, len(data), taille_segment):
        segment = data[i:i + taille_segment]
        segment_path = os.path.join(dossier_sortie, f'segment_{i // taille_segment + 1}.json')
        
        # Écrire chaque segment dans un fichier
        with open(segment_path, 'w', encoding='utf-8') as segment_file:
            json.dump(segment, segment_file, indent=4)
    
    print(f"Fichier JSON segmenté en {nombre_parties} parties dans le dossier '{dossier_sortie}'.")


In [ ]:
def assembler_json(dossier_segments, fichier_sortie):
    """
    Assemble plusieurs fichiers JSON en un seul fichier et supprime les segments.

    :param dossier_segments: Dossier contenant les segments JSON.
    :param fichier_sortie: Chemin du fichier JSON de sortie.
    """
    fichiers = sorted(os.listdir(dossier_segments))  # Trier les segments par nom
    data_combinee = []

    for fichier in fichiers:
        segment_path = os.path.join(dossier_segments, fichier)
        with open(segment_path, 'r', encoding='utf-8') as segment_file:
            data_combinee.extend(json.load(segment_file))

    # Écrire les données combinées dans un seul fichier JSON
    with open(fichier_sortie, 'w', encoding='utf-8') as output_file:
        json.dump(data_combinee, output_file, indent=4)
    
    print(f"Segments JSON assemblés dans '{fichier_sortie}', et les segments supprimés.")